<a href="https://colab.research.google.com/github/srijan0709/Rough-GPT/blob/main/RoughGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## Downloading required packages
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.3 MB/s eta 0:0

In [4]:
### Importing required libraries

from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone ##pinecone for storing our vector embeddings
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
!mkdir Ug_rulebook ##creating directory

In [7]:
loader = PyPDFDirectoryLoader("Ug_rulebook")
data = loader.load()  ## loading data

In [8]:
data

[Document(page_content='1  \n \n \n \n \n \n \nApplicable to the B.Tech., B.S., B.Des.,   \nDual Degree students admitted from the  \nAcademic Year 2007 - 2008 \n \n \n \n \n \n \n \nUpdated: May, 2023  ', metadata={'source': 'Ug_rulebook/ugrulebook.pdf', 'page': 0}),
 Document(page_content='2 Rules are classified into three separate categories as follows: (I) those which may be implemented \nwithin a department by DUGC/DPGC, (ii) those that require a decision at the level of A/Dean AP, or \nUGAPEC/PGAPEC, based on recommendations from the department bodies (iii) those that need to \nbe discussed in the Senate for a decision. \n \nTherefore, rules are colored with one of three colors.  \n1. The color green indicates that the final authority for rule is the Convener DUGC \n2. The color yellow, and underlined means that the final authority is Associate Dean (AP)/Dean \n(AP). \n3. The color yellow, without an underline means that the Convener, UGAPEC is the authority. \n4. The color blue 

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20) ##splitting the data into chunks

In [10]:
docs = text_splitter.split_documents(data)

In [11]:
len(docs)

261

In [12]:
docs[260]

Document(page_content='SPI  : Semester Performance Index which is obtained by dividing the Se-\nmester Grade Points by the Semester Credits. \n \nSummer Term  : Approximately of 8 weeks duration from May to July. \n \nTranscript  : Consolidated statement of the Academic Performance of a student \nfor all the semesters completed. \n \nUGAPEC  : Under-Graduate Academic Performance Evaluation Committee \n \nUGPC  : Under-Graduate Programmes Committee \n \nURA  : Undergraduate Research Award. \n \n \n-------xxxx------', metadata={'source': 'Ug_rulebook/ugrulebook.pdf', 'page': 45})

In [13]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') ## downloading embeddings


In [14]:
query_result = embeddings.embed_query("I am Srijan")

In [15]:
len(query_result)

384

In [16]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '9ad66fd9-5adb-44ea-92a0-fa68f53c1291')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [17]:
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "roughgpt"

In [18]:
#docsearch=Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

In [28]:
## loading our saved embeddings from pinecone
docsearch = Pinecone.from_existing_index(index_name, embeddings)


In [29]:
query = "What is the IDDDP program"

ans = docsearch.similarity_search(query, k=2) ## this function will serch and retur the top two paragraph similar to question

In [21]:
ans

[Document(page_content='IDDDP, (with/without HONORS) which may differ depending on the parent academic unit of \nthe incoming student to facilitate introduction considering (a). \ne) The DUGC/DPGC of the admitting academic unit must prescribe beforehand and certify the \ncompletion of the HONORS requirement. An academic unit can admit at most 2 UG students in \neach DD/ M.Tech. specialization.  \nf) The selection and entry of all candidates in IDDDP will remain provisional till the successful'),
 Document(page_content='IDDDP, (with/without HONORS) which may differ depending on the parent academic unit of \nthe incoming student to facilitate introduction considering (a). \ne) The DUGC/DPGC of the admitting academic unit must prescribe beforehand and certify the \ncompletion of the HONORS requirement. An academic unit can admit at most 2 UG students in \neach DD/ M.Tech. specialization.  \nf) The selection and entry of all candidates in IDDDP will remain provisional till the successful')

In [22]:
from huggingface_hub import notebook_login
import torch


In [24]:
notebook_login()

In [25]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [26]:
!pip install accelerate
!pip install bitsandbytes




import bitsandbytes
# from bnb.nn import Linear8bitLt

import accelerate


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [27]:
## creating the pipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [49]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

In [50]:
## creating prompt for LLAMA 2

SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS
instruction = """
{context}

Question: {question}
"""
template = B_INST + SYSTEM_PROMPT + instruction + E_INST
template

"[INST]<>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [51]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])


In [52]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 4}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)


In [53]:
result = qa_chain("What is the IDDDP program")


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [54]:
result['result']

'  Based on the provided context, the IDDDP program appears to be an Interdisciplinary Dual Degree Programme.'

In [55]:
result1 = qa_chain("How to calculate SPI")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [37]:
result1['result']

'  Based on the given context, the formula to calculate the SPI (Cumulative Performance Index) is:\n\nSPI = (C1 x g1 + C2 x g2 + C3 x g3 + C4 x g4 + C5 x g5) / (C1 + C2 + C3 + C4 + C5)\n\nWhere:\n\n* C1, C2, C3, C4, and C5 are the credits of the courses taken by the student in a semester.\n* g1, g2, g3, g4, and g5 are the grade points earned by the student in those courses.\n\nThe SPI is calculated to two decimal places and takes into consideration the FR (Final Result) of the semester. The updated SPIs will be shown in the transcript only after the last date of grade submission (course + project) as per the Academic Calendar.'

In [38]:
result2  = qa_chain("What is the role of a faculty advisor")

In [39]:
result2['result']

'  Based on the provided context, the role of a faculty advisor is to:\n\n1. Guide students about the rules and regulations governing the courses of study.\n2. Advise students to register for courses as per the curriculum and their academic standing.\n\nIt is important to note that the faculty advisor is the person that parents/guardians should contact for performance-related issues of their wards.'

In [40]:
result3  = qa_chain("what is the duration of DDP")
result3['result']

'  Based on the provided context, the duration of Dual Degree Projects (DDPs) is 14 months.'

In [41]:
result4 = qa_chain("who to consult before dropping any course")
result4['result']

'  Based on the given context, the answer to the question "Who to consult before dropping any course?" is:\n\na. Faculty Adviser\n\nAccording to the context, a student can drop a semester due to severe health problems or other valid reasons beyond their control, in consultation with the Faculty Adviser and with permission from the DUGC/UGAPEC. Therefore, the first option, "Faculty Adviser," is the correct answer.'

In [42]:
result5 = qa_chain("who will evaluate my dual degree project")
result5['result']

'  Based on the context provided, it appears that the evaluation of the Dual Degree Project will be done by a committee. Specifically, the question states that the committee evaluating the Final Stage of the Dual Degree Project of a given Dual Degree student will believe that the student has carried out truly exceptional work. Therefore, the answer to your question is that the evaluation of your Dual Degree Project will be done by a committee.'

In [43]:
result6 = qa_chain("what is the cpi requirements for branch change")
result6['result']

'  Based on the given context, the CPI requirements for branch change are:\n\nFor a student to be eligible for a branch change, they must have either:\n\n(I) a higher "Branch-Change-CPI" or\n(II) the same "Branch-Change-CPI" and a higher "overall-CPI".\n\nIn case there is a tie for both the "Branch-Change-CPI" and the "overall-CPI", the following will apply:\n\nIf a student with "Branch-Change-CPI" x and "overall-CPI" y is shifted from branch A to branch B, then every student with "Branch-Change-CPI" x and "overall-CPI" y with a request for branch B will be shifted to branch B, even if their request violates (ii) above. If necessary, extra seats will be created to accommodate them.\n\nTherefore, the CPI requirements for branch change are:\n\n* Either a higher "Branch-Change-CPI" or the same "Branch-Change-CPI" and a higher "overall-CPI".\n* In case of a tie, the student with the highest "Branch-Change-CPI" will be given priority.'

In [44]:
result7 = qa_chain("what is the minimum credit requirement for minor degree?")
result7['result']

'  Thank you for providing the context. Based on the information provided, the minimum credit requirement for a minor degree is 30 credits.'

In [45]:
result8 = qa_chain("how many students take admission in IIT bombay each year?")
result8['result']

'  Based on the provided context, I can answer the question as follows:\n\nAccording to the context, IIT Bombay on an average annually admits more than 1000 candidates for the undergraduate programs (B.Tech./ Dual Degree and B.S.) through the Joint Entrance Examination (JEE). Additionally, the context mentions that around 30 candidates take admission in the B.Des. Programme through the Undergraduate Common Entrance Exam for Design (UCEED), around 300 candidates for the M.Sc. and the M.Sc. Ph.D. Dual Degree programs, more than 1000 candidates for postgraduate programs, and around 300 candidates for Ph.D. programs.\n\nTherefore, the total number of students who take admission in IIT Bombay each year is:\n\n1000 + 30 + 300 + 1000 + 300 = 2630\n\nSo, the answer to the question is 2630 students take admission in IIT Bombay each year.'

In [46]:
result9 = qa_chain("how many students take admission in B.tech programmes at IIT Bombay?")
result9['result']

"  Based on the provided context, the number of students who take admission in B.Tech programs at IIT Bombay is not explicitly mentioned. However, it is mentioned that IIT Bombay on an average annually admits more than 1000 candidates for the undergraduate programs (B.Tech./ Dual Degree and B.S.) through the Joint Entrance Examination (JEE). Therefore, we can infer that the number of students who take admission in B.Tech programs at IIT Bombay is likely to be around 1000 or more. However, I don't know the exact number, so I can't provide a definitive answer."

In [58]:
# !pip install tensorflow==2.14.0

## FOR DEPLOYING ON GRADIO AND TAKING FEEDBACKS

In [59]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding
# !pip install gradio -q

In [60]:
# import gradio as gr

In [62]:
# def chatgpt_clone(input,chatbot):


#     output = qa_chain(input)
#     output1 = output['result']
#     chatbot.append((input, output1))

#     return output1,chatbot


# block = gr.Blocks()


# with block:
#     gr.Markdown("""<h1><center>RoughGPT</center></h1>
#     """)
#     chatbot = gr.Chatbot()
#     message = gr.Textbox(placeholder=prompt)
#     state = gr.State()
#     submit = gr.Button("SEND")
#     submit.click(chatgpt_clone, [message, chatbot], [message, chatbot])

# block.launch(debug = True)

# with gr.Blocks() as demo:

#     chatbot = gr.Chatbot(label='GPT', height=750)
#     msg = gr.Textbox()
#     clear = gr.ClearButton([msg, chatbot])

#     msg.submit(chatgpt_clone, [msg, chatbot], [msg, chatbot])

# demo.launch()

# def chatgpt_clone(input, history):



#     output = qa_chain(input)
#     output = output['result']

#     history.append((input, output))
#     return history, history


# block = gr.Blocks()


# with block:
#     gr.Markdown("""<h1><center>Build Yo'own ChatGPT with OpenAI API & Gradio</center></h1>
#     """)
#     chatbot = gr.Chatbot()
#     message = gr.Textbox(placeholder=prompt)
#     state = gr.State()
#     submit = gr.Button("SEND")
#     submit.click(chatgpt_clone, inputs=[message, state], outputs=[chatbot, state])

# block.launch(debug = True)



In [61]:
# def gpt_clone(input, history):

#     history = history or []
#     s = list(sum(history, ()))
#     s.append(input)
#     inp = ' '.join(s)

#     output = qa_chain(input)
#     output = output['result']

#     history.append((input, output))
#     return history, history

# block = gr.Blocks()


# with block:
#     gr.Markdown("""<h1><center>RoughGPT</center></h1>
#     """)
#     chatbot = gr.Chatbot()
#     message = gr.Textbox(placeholder=prompt)
#     state = gr.State()
#     submit = gr.Button("SEND")
#     submit.click(gpt_clone, inputs=[message, state], outputs=[chatbot, state])

# block.launch(debug = True)